In [ ]:
import datetime

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
sns.set_theme(style="whitegrid", palette="deep")
task_palette = sns.color_palette("deep")
plt.rcParams["text.usetex"] = True
plt.rcParams.update({"figure.titlesize": "small"})
plt.rcParams.update({"axes.titlesize": "small"})
plt.rcParams.update({"axes.labelsize": "small"})
plt.rcParams.update({"ytick.labelsize": "small"})
plt.rcParams.update({"xtick.labelsize": "small"})
plt.rcParams.update({"legend.fontsize": "small"})

In [ ]:
path = "./results/results-mcs_oracles-2024-05-18_14-26-47.csv"
df = pd.read_csv(path, sep=";")
df

In [ ]:
df.groupby("taskset_position")["is_safe"].apply(lambda x: (x == x.iloc[0]).all()).all()

In [ ]:
df["tid"] = df["taskset_position"]
df["schedulable"] = df["is_safe"]
df["Schedulable"] = df["is_safe"]
df["n_task"] = df["nbt"]
df["Number of tasks"] = df["nbt"]
df["duration_s"] = df["duration_ns"] / 1e9
df["n_visited"] = df["visited_count"]

In [ ]:
df["safe_oracles_fmt"] = df["safe_oracles"].str.strip("[']")
df["unsafe_oracles_fmt"] = df["unsafe_oracles"].str.strip("[']")
df["safe_oracles_fmt"] = df["safe_oracles_fmt"].apply(lambda x: x if len(x) else None)
df["unsafe_oracles_fmt"] = df["unsafe_oracles_fmt"].apply(
    lambda x: x if len(x) else None
)

df["oracle"] = df.apply(
    lambda x: (
        x["safe_oracles_fmt"]
        if x["safe_oracles_fmt"]
        else x["unsafe_oracles_fmt"]
        if x["unsafe_oracles_fmt"]
        else "none"
    ),
    axis=1,
)

In [ ]:
df["oracle"] = df.apply(
    lambda x: (
        x["safe_oracles_fmt"]
        if x["safe_oracles_fmt"]
        else x["unsafe_oracles_fmt"]
        if x["unsafe_oracles_fmt"]
        else "none"
    ),
    axis=1,
)

In [ ]:
df_root = df[["tid", "n_task", "schedulable", "Number of tasks", "Schedulable"]]
df_root = df_root.set_index("tid")

In [ ]:
df_root["schedulable"].value_counts() / 6

In [ ]:
df_ss = df.pivot(index="tid", columns="oracle", values=["duration_s", "n_visited"])
df_ss.columns = list(map(lambda x: "_".join(x), df_ss.columns))
df_ss = df_root.join(df_ss)

In [ ]:
oracle_sort_list = (
    df.groupby("oracle")["n_visited"].median().sort_values().index.to_list()[::-1]
)

In [ ]:
df_comp = df.loc[df["oracle"] != "none"].merge(
    df.loc[df["oracle"] == "none", ["tid", "duration_s", "n_visited"]],
    on="tid",
    suffixes=("", "_none"),
)
df_comp["n_visited_ratio"] = df_comp["n_visited"] / df_comp["n_visited_none"]
df_comp["duration_s_ratio"] = df_comp["duration_s"] / df_comp["duration_s_none"]
df_comp["n_visited_change"] = df_comp["n_visited_ratio"] - 1
df_comp["duration_s_change"] = df_comp["duration_s_ratio"] - 1

In [ ]:
df_comp

In [ ]:
df_comp_melt = df_comp.melt(
    id_vars=["tid", "oracle", "schedulable"],
    value_vars=["n_visited_change", "duration_s_change"],
)


df_comp_melt = df_comp_melt.rename(
    columns={
        "variable": "Dimension",
        "value": "Relative change",
        "schedulable": "Schedulable",
        "oracle": "Oracle",
    }
)

df_comp_melt["Relative change"] = df_comp_melt["Relative change"] * 100

metric_map = {
    "duration_s_change": "Execution time",
    "n_visited_change": "# of visited states",
}

df_comp_melt["Dimension"] = df_comp_melt["Dimension"].map(lambda x: metric_map[x])

oracle_map = {
    "hi-idle-point": "HI idle point",
    "over-demand": "Over demand",
    "none": "None",
    "negative-laxity": "Negative laxity",
    "hi-over-demand": "HI over demand",
    "negative-worst-laxity": "Negative worst laxity",
}

df_comp_melt["Oracle"] = df_comp_melt["Oracle"].map(
    lambda x: oracle_map[x] if x in oracle_map else x
)

oracle_scope = [
    "HI idle point",
    "Over demand",
    "HI over demand",
    "Negative laxity",
    "Negative worst laxity",
    #    "Sum-min-laxity",
    #    "Sum-min-worst-laxity",
]

df_comp_melt = df_comp_melt.loc[df_comp_melt["Oracle"].isin(oracle_scope)]

s = df_comp_melt["Dimension"] == "Execution time"
df_order = (
    df_comp_melt.loc[s]
    .groupby(["Oracle", "Schedulable"], as_index=False)["Relative change"]
    .median()
)
oracles_order = (
    df_order.loc[df_order["Schedulable"]].set_index("Oracle")["Relative change"]
    + df_order.loc[~df_order["Schedulable"]].set_index("Oracle")["Relative change"]
)
oracles_order = oracles_order.sort_values(ascending=False).index.tolist()

oracles_order = [
    "HI idle point",
    "Negative laxity",
    #    "Sum-min-laxity",
    "Negative worst laxity",
    #    "Sum-min-worst-laxity",
    "Over demand",
    "HI over demand",
]

df_comp_melt["Schedulable"] = df_comp_melt["Schedulable"].map(
    lambda x: "Schedulable" if x else "Unschedulable"
)

In [ ]:
df_comp_melt

In [ ]:
def fmt_percent(x, *args):
    return ("+" if x > 0 else "") + rf"{x:.0f}\%"


def fmt_percent_1f(x, *args):
    return ("+" if x > 0 else "") + rf"{x:.0f}\%"

In [ ]:
s = df_comp_melt["Dimension"] == "# of visited states"
df_comp_melt.loc[s, "Dimension"] = r"\# of visited states"

In [ ]:
fg = sns.catplot(
    df_comp_melt,
    kind="bar",
    y="Relative change",
    x="Oracle",
    col="Dimension",
    hue="Schedulable",
    height=5.5129 / 2,
    aspect=1,
    sharex=False,
    col_order=[r"\# of visited states", "Execution time"],
    errorbar=None,
    estimator="median",
    order=oracles_order,
    facet_kws={"gridspec_kws": {"wspace": 0.1}},
)


fg.set_titles("{col_name}")

# draw a dashed vertical line at 0 on the first plot of the face grid
fg.axes[0, 1].axhline(0, color="k", linestyle="--")
fg.axes[0, 0].axhline(0, color="k", linestyle="--")

fg.axes[0, 0].bar_label(
    fg.axes[0, 0].containers[0], fontsize=8, fmt=fmt_percent_1f, rotation=45
)
fg.axes[0, 0].bar_label(
    fg.axes[0, 0].containers[1], fontsize=8, fmt=fmt_percent_1f, rotation=45
)
fg.axes[0, 1].bar_label(
    fg.axes[0, 1].containers[0], fontsize=8, fmt=fmt_percent_1f, rotation=45
)
fg.axes[0, 1].bar_label(
    fg.axes[0, 1].containers[1], fontsize=8, fmt=fmt_percent_1f, rotation=45
)

fg.axes[0, 0].yaxis.set_major_formatter(fmt_percent)

fg.axes[0, 1].set_ylim(-110, 20)

fg.axes[0, 0].set_xlabel(None)
fg.axes[0, 1].set_xlabel(None)

for ax in fg.axes.flat:
    for k in range(len(ax.containers)):
        h = ax.patches[k].get_height()
        if h >= 0.8:
            ax.patches[k].set_hatch("*")
            ax.patches[k].set_edgecolor("k")


sns.despine(fg.figure, bottom=True)

fg.set_xticklabels(rotation=90)

for ax in fg.axes.flat:
    for i, cnt in enumerate(ax.containers):
        for artist in cnt:
            if i == 0:
                artist.set_edgecolor(task_palette[0])
                artist.set_facecolor("white")
                artist.set_hatch("//")

# sns.move_legend(fg, "lower left", bbox_to_anchor=(0.25, 1.1), ncol=2, title=None)

circ1 = mpatches.Patch(
    hatch="//", label="Unschedulable", edgecolor=task_palette[0], facecolor="white"
)
circ2 = mpatches.Patch(facecolor=task_palette[1], hatch="", label="Schedulable")
new_handles = [circ1, circ2]

sns.move_legend(
    fg,
    "lower left",
    bbox_to_anchor=(0.0, -0.45),
    ncol=1,
    title=None,
    handles=new_handles,
)

plt.suptitle(
    r"$P_{\mathsf{HI}} = 0.5, T^{\mathsf{min}} = 5, T^{\mathsf{max}} = 30, n = 5, U^* \in [0.8;1;0.01]$",
    y=1.05,
    x=0.46,
)

In [ ]:
fg.savefig(
    f"./fig_rtss/{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_oracle.pdf",
    bbox_inches="tight",
)
fg.savefig(
    "./fig_rtss/oracle.pdf",
    bbox_inches="tight",
)